In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import chi2,SelectKBest

In [3]:
titanic = pd.read_csv('./train_titanic.csv')

In [9]:
titanic = titanic.drop(labels=['Name','PassengerId','Ticket','Fare','Cabin'],axis=1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(titanic.iloc[:,1:],titanic.iloc[:,0],test_size=0.3,random_state=10)

In [15]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         113
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [18]:
### Imputation Transformer
trf1 = ColumnTransformer([('impute_age',SimpleImputer(),[2]),
                         ('impute_embarked',SimpleImputer(strategy='most_frequent'),[5])],
                         remainder='passthrough'
                        )

In [22]:
### encoding
trf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,5])],remainder='passthrough')

In [23]:
### Scaling
trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,9))])

In [24]:
### Feature Selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [25]:
### Model Training
trf5 = DecisionTreeClassifier()

### Creating Pipelines

In [26]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [29]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 5])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 9, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001FA0B72A550>)),
                ('trf5', DecisionTreeClassifier())])

In [47]:
### Points to decode a pipeline


pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [28]:
from sklearn import set_config
set_config(display='diagram')

In [30]:
y_pred = pipe.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6231343283582089

### Hyper Parameter Tuning in Pipelines

In [32]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [35]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [5])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          5])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 9, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001FA0B72A550>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [36]:
grid.best_score_

0.6149161290322581

In [37]:
grid.best_params_

{'trf5__max_depth': 3}

### Exporting Pipelines

In [38]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [39]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S
